# OPE Experiment with Open Bandit Dataset
---
This notebook demonstrates an example of conducting OPE of Bernoulli Thompson Sampling (BernoulliTS) as an evaluation policy using some OPE estimators and logged bandit feedback generated by running the Random policy (behavior policy) on the ZOZOTOWN platform.

In [1]:
from sklearn.linear_model import LogisticRegression

# import open bandit pipeline (obp)
import obp
from obp.dataset import OpenBanditDataset
from obp.policy import BernoulliTS
from obp.ope import (
    OffPolicyEvaluation, 
    RegressionModel,
    InverseProbabilityWeighting as IPS,
    DirectMethod as DM,
    DoublyRobust as DR,
)

In [2]:
# obp version
print(obp.__version__)

0.5.1


## (1) Data Loading and Preprocessing

`obp.dataset.OpenBanditDataset` is an easy-to-use data loader for Open Bandit Dataset. 

It takes behavior policy ('bts' or 'random') and campaign ('all', 'men', or 'women') as inputs and provides dataset preprocessing.

In [3]:
# When `data_path` is not given, this class downloads the small-sized version of Open Bandit Dataset.
dataset = OpenBanditDataset(behavior_policy='random', campaign='all')

INFO:obp.dataset.real:When `data_path` is not given, this class downloads the example small-sized version of the Open Bandit Dataset.
/Users/usaito/.pyenv/versions/3.9.5/envs/zr-obp/lib/python3.9/site-packages/obp-0.5.1-py3.9.egg/obp/dataset/real.py:203: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  item_feature_cat = self.item_context.drop("item_feature_0", 1).apply(
/Users/usaito/.pyenv/versions/3.9.5/envs/zr-obp/lib/python3.9/site-packages/obp-0.5.1-py3.9.egg/obp/dataset/real.py:206: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  self.action_context = pd.concat([item_feature_cat, item_feature_0], 1).values


In [4]:
# obtain logged bandit data generated by behavior policy
bandit_data = dataset.obtain_batch_bandit_feedback()

# `bandit_data` is a dictionary storing logged bandit feedback
bandit_data.keys()

dict_keys(['n_rounds', 'n_actions', 'action', 'position', 'reward', 'pscore', 'context', 'action_context'])

### let's see some properties of the dataset class

In [5]:
# name of the dataset is 'obd' (open bandit dataset)
dataset.dataset_name

'obd'

In [6]:
# number of actions of the "All" campaign is 80
dataset.n_actions

80

In [7]:
# small example dataset has 10,000 rounds
dataset.n_rounds

10000

In [8]:
# default context (feature) engineering creates context vector with 20 dimensions
dataset.dim_context

20

In [9]:
# ZOZOTOWN recommendation interface has three positions
# (please see https://github.com/st-tech/zr-obp/blob/master/images/recommended_fashion_items.png)
dataset.len_list

3

## (2) Production Policy Replication

After preparing the dataset, we now replicate the BernoulliTS policy implemented on the ZOZOTOWN recommendation interface during the data collection period.

Here, we use `obp.policy.BernoulliTS` as an evaluation policy. 
By activating its `is_zozotown_prior` argument, we can replicate (the policy parameters of) BernoulliTS used in ZOZOTOWN production.

In [10]:
evaluation_policy = BernoulliTS(
    n_actions=dataset.n_actions, # number of actions; |A|
    len_list=dataset.len_list, # number of items in a recommendation list; K
    is_zozotown_prior=True, # replicate the BernoulliTS policy in the ZOZOTOWN production
    campaign="all",
    random_state=12345,
)

In [11]:
# compute the action choice probabilities of the evaluation policy using Monte Carlo simulation
action_dist = evaluation_policy.compute_batch_action_dist(
    n_sim=100000, n_rounds=bandit_data["n_rounds"],
)

In [12]:
# action_dist is an array of shape (n_rounds, n_actions, len_list) 
# representing the distribution over actions by the evaluation policy
action_dist[:5]

array([[[0.01078, 0.00931, 0.00917],
        [0.00167, 0.00077, 0.00076],
        [0.0058 , 0.00614, 0.00631],
        ...,
        [0.0008 , 0.00087, 0.00071],
        [0.00689, 0.00724, 0.00755],
        [0.0582 , 0.07603, 0.07998]],

       [[0.01078, 0.00931, 0.00917],
        [0.00167, 0.00077, 0.00076],
        [0.0058 , 0.00614, 0.00631],
        ...,
        [0.0008 , 0.00087, 0.00071],
        [0.00689, 0.00724, 0.00755],
        [0.0582 , 0.07603, 0.07998]],

       [[0.01078, 0.00931, 0.00917],
        [0.00167, 0.00077, 0.00076],
        [0.0058 , 0.00614, 0.00631],
        ...,
        [0.0008 , 0.00087, 0.00071],
        [0.00689, 0.00724, 0.00755],
        [0.0582 , 0.07603, 0.07998]],

       [[0.01078, 0.00931, 0.00917],
        [0.00167, 0.00077, 0.00076],
        [0.0058 , 0.00614, 0.00631],
        ...,
        [0.0008 , 0.00087, 0.00071],
        [0.00689, 0.00724, 0.00755],
        [0.0582 , 0.07603, 0.07998]],

       [[0.01078, 0.00931, 0.00917],
        [0.0016

## (3) Off-Policy Evaluation (OPE)

The next step is **OPE**, which attempts to estimate the performance of new decision making policies using only log data generated by behavior, past policies. 

Here, we use
- **Inverse Propensity Score (IPS)**
- **DirectMethod (DM)**
- **Doubly Robust (DR)**

to estimate the performance of Bernoulli TS using only the log data. 

### (3-1) obtain a reward estimator
`obp.ope.RegressionModel` simplifies the process of reward modeling

$r(x,a) = \mathbb{E} [r \mid x, a] \approx \hat{r}(x,a)$

In [13]:
# obp.ope.RegressionModel
regression_model = RegressionModel(
    n_actions=dataset.n_actions, # number of actions; |A|
    len_list=dataset.len_list, # number of items in a recommendation list; K
    base_model=LogisticRegression(C=100, max_iter=10000, random_state=12345), # any sklearn classifier
)

In [14]:
estimated_rewards = regression_model.fit_predict(
    context=bandit_data["context"],
    action=bandit_data["action"],
    reward=bandit_data["reward"],
    position=bandit_data["position"],
    random_state=12345,
)

In [15]:
estimated_rewards[:, :, 0] # \hat{q}(x,a)

array([[1.58148566e-04, 1.68739271e-02, 1.59238728e-04, ...,
        1.62785786e-04, 1.58892825e-04, 1.39337751e-04],
       [9.43487804e-05, 1.01350576e-02, 9.49991949e-05, ...,
        9.71154498e-05, 9.47928215e-05, 8.31259330e-05],
       [9.69432542e-08, 1.05192815e-05, 9.76116177e-08, ...,
        9.97862793e-08, 9.73995491e-08, 8.54108347e-08],
       ...,
       [1.49986944e-04, 1.60169285e-02, 1.51020855e-04, ...,
        1.54384887e-04, 1.50692800e-04, 1.32146777e-04],
       [3.99016414e-04, 4.15165918e-02, 4.01766279e-04, ...,
        4.10713441e-04, 4.00893762e-04, 3.51565900e-04],
       [3.27203945e-04, 3.42986101e-02, 3.29459070e-04, ...,
        3.36796524e-04, 3.28743530e-04, 2.88290813e-04]])

### (3-2) OPE
`obp.ope.OffPolicyEvaluation` simplifies the OPE process

$V(\pi_e) \approx \hat{V} (\pi_e; \mathcal{D}_0, \theta)$ using DM, IPS, and DR

In [16]:
ope = OffPolicyEvaluation(
    bandit_feedback=bandit_data, # bandit data
    ope_estimators=[
        IPS(estimator_name="IPS"), 
        DM(estimator_name="DM"), 
        DR(estimator_name="DR"),
    ] # used estimators
)

In [17]:
estimated_policy_value = ope.estimate_policy_values(
    action_dist=action_dist, # \pi_e(a|x)
    estimated_rewards_by_reg_model=estimated_rewards, # \hat{q}
)

In [18]:
# OPE results given by the three estimators
estimated_policy_value

{'IPS': 0.00455288, 'DM': 0.004729396841185728, 'DR': 0.0047693518747239025}

## (4) Evaluation of OPE

Our final step is the **evaluation of OPE**, which evaluates the OPE performance (estimation accuracy) of the OPE estimators.

Specifically, we asses the accuracy of the estimators by comparing their estimation with the ground-truth policy value estimated via the on-policy estimation from Open Bandit Dataset.

This type evaluation of OPE is possible, because Open Bandit Dataset contains a set of *multiple* different logged bandit datasets collected by running different policies on the same platform at the same time.

### (4-1) Approximate the Ground-truth Policy Value
$V(\pi) \approx \frac{1}{|\mathcal{D}_{te}|} \sum_{i=1}^{|\mathcal{D}_{te}|} r_i, $

In [19]:
# we first calculate the ground-truth policy value of the evaluation policy
# , which is estimated by averaging the factual (observed) rewards contained in the dataset (on-policy estimation)
policy_value_bts = OpenBanditDataset.calc_on_policy_policy_value_estimate(
    behavior_policy='bts', campaign='all'
)

INFO:obp.dataset.real:When `data_path` is not given, this class downloads the example small-sized version of the Open Bandit Dataset.
/Users/usaito/.pyenv/versions/3.9.5/envs/zr-obp/lib/python3.9/site-packages/obp-0.5.1-py3.9.egg/obp/dataset/real.py:203: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  item_feature_cat = self.item_context.drop("item_feature_0", 1).apply(
/Users/usaito/.pyenv/versions/3.9.5/envs/zr-obp/lib/python3.9/site-packages/obp-0.5.1-py3.9.egg/obp/dataset/real.py:206: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  self.action_context = pd.concat([item_feature_cat, item_feature_0], 1).values


### (4-2) Evaluation of OPE
Now, let's evaluate the OPE performance (estimation accuracy) of the three estimators 

$SE (\hat{V}; \mathcal{D}_0) := \left( V(\pi_e) - \hat{V} (\pi_e; \mathcal{D}_0, \theta) \right)^2$,     (squared error of $\hat{V}$)

In [20]:
squared_errors = ope.evaluate_performance_of_estimators(
    ground_truth_policy_value=policy_value_bts,
    action_dist=action_dist,
    estimated_rewards_by_reg_model=estimated_rewards,
    metric="se", # squared error
)

In [21]:
squared_errors # IPS is the most accurate 

{'IPS': 1.245242943999999e-07,
 'DM': 2.8026101545742736e-07,
 'DR': 3.241615572516226e-07}

We can iterate the above process several times and calculate the following MSE

$MSE (\hat{V}) := T^{-1} \sum_{t=1}^T SE (\hat{V}; \mathcal{D}_0^{(t)}) $

where $\mathcal{D}_0^{(t)}$ is the synthetic data in the $t$-th iteration

Note that the OPE demonstration here is with the small size example version of our dataset. 
Please use its full size version (https://research.zozo.com/data.html) to produce more reasonable results.